In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
indexing_distinct_tables = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [2]:
tf.__version__

'2.0.0-rc0'

In [3]:
test_articles = pd.read_csv('../dataset/data_articles_test.csv', delimiter=',')

In [4]:
# embedding_model = gs.models.FastText.load_fasttext_format('pre_trained_models/cc.en.300.bin')
embedding_model = gs.models.FastText.load('../train_embedding_models/fasttext_embedding_50d_all_signals')

In [5]:
indexing_distinct_tables.indices.close(index='distinct_tables')
indexing_distinct_tables.indices.put_settings(index='distinct_tables', body={"index": {"similarity": {"default": {"type": "classic"}}}})
indexing_distinct_tables.indices.open(index='distinct_tables')

{'acknowledged': True, 'shards_acknowledged': True}

In [6]:
# "tablePgID"
# "tablePgTitle"
# "tablePgFullText"
# "tablePgMetaDescription"
# "tablePgSummary"
# "tablePgKeywords"
# "tableSectionTitle"
# "tableCaption"
# "tableHeader"
# "tableBody"

In [7]:
def search_indexing(query):
    
    result= indexing_distinct_tables.search(
        index="distinct_tables", 
        body = {
        "_source": ["tablePgID","tablePgTitle"],
        "from" : 0,
        "size" : 100,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    query, 
              "fields": ["tablePgFullText"] 
            }
        }
    })
    
    return result

In [8]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID[0] == ID_goal:
    
            accuracy = 1
            break;

    return accuracy

In [9]:
MAX_PAD = 31

def sequence_padding(X_DIM, value):
    
    value_padding = np.pad(value, ((0,MAX_PAD - X_DIM),(0,0)), 'constant')
    
    return value_padding

In [10]:
def create_embedding(value):

    value = tknzr.tokenize(str(value))
    
    if len(value) < MAX_PAD:
        
        embedding = embedding_model.wv[value]
        
        padding_embedding = sequence_padding(embedding.shape[0],embedding)
        
        return padding_embedding
        
    else:
        
        embedding = embedding_model.wv[value[0:MAX_PAD]]
        
        return embedding

In [11]:
def search_index(query):
    
    tables_index = []

    result_index = search_indexing(query)
        
    for hit in result_index['hits']['hits']:
    
        table_ID = hit['_source']['tablePgID']
        
        table_page_title = hit['_source']['tablePgTitle']
    
        tables_index.append([table_ID,table_page_title])
    
    return tables_index

In [12]:
ranking_model = tf.keras.models.load_model('context_plus_single_attention_model_title_10_0.9459.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
test_articles.head(1)

,full_text,keywords,meta_description,meta_keywords,page_id,page_title,summary,tags
0,when comes pioneers progressive rock handful b...,faces anderson album chris jon went record tre...,yes among year rock roll hall fame inductees c...,NaN,4125505,rock roll hall famers jon anderson trevor rabi...,because good photos the best yes album coverss...,NaN


In [14]:
result = []

def run_search(k):
    
    TOP_K = k
    accuracy = []
    
    for i, row in tqdm(test_articles.iterrows()):
        
        article_ID = row['page_id']
        article_title_text = row['page_title']
        query = row['page_title']+" "+row['meta_description']+" "+row['keywords']
        
        article_title = []
        title_table = []
        ranked_tables_model = []

        ranked_tables_index = search_index(query)
        
        if len(ranked_tables_index) > 0:
            
            article_title_embedding = create_embedding(article_title_text)
            
            for table_ID, table_title in (ranked_tables_index):
                    
                table_title_embedding = create_embedding(str(table_title))

                article_title.append(article_title_embedding)
                title_table.append(table_title_embedding)

            article_title = np.array(article_title)
            title_table = np.array(title_table)
           
            table_ranking_model = ranking_model.predict([article_title,title_table])

            for i in range(0,len(table_ranking_model)):

                ranked_tables_model.append([ranked_tables_index[i][0],ranked_tables_index[i][1],table_ranking_model[i][0]]) 

            data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
            data_frame_sorting = data_frame.sort_values('table_ranking', ascending=False)   
#             final_ranked_tables = data_frame_sorting.iloc[0:TOP_K,0:1].values
            
            selected_top = data_frame_sorting.head(TOP_K)
            min_score = selected_top['table_ranking'].min()
            draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] >= min_score]
            final_ranked_tables = draw_tables_socres.iloc[:,0:1].values
        
            accuracy.append(get_accuracy(article_ID, final_ranked_tables))

    result.append(["Acc@"+str(k),str(round(np.mean(accuracy),4))])

In [15]:
accuracy_K = [1,5,10,20]

for k in accuracy_K:
     
    run_search(k)

1026it [00:59, 17.29it/s]
1026it [00:57, 18.41it/s]
1026it [00:57, 17.75it/s]
1026it [00:58, 18.44it/s]


In [16]:
result

[['Acc@1', '0.3177'],
 ['Acc@5', '0.3801'],
 ['Acc@10', '0.424'],
 ['Acc@20', '0.4756']]